In [ ]:
# CAENDigitizerControl - Written by Edgar Mao
#
# Adapted from demo_scope.py by Giovanni Cerretani

In [ ]:
import numpy as np

from caen_felib import device, error
import matplotlib.pyplot as plt
import caen_felib
from ctypes import *

In [ ]:
# Digitizer settings

# Runtime
runtime = 10000
filesize = 1000

# Board Configuration parameters
reclen = 4096*4             # record length, in units of ns
post_trigger = 4096*2       # ns
trig_edge = f"RISE"         # Self-trigger edge, "RISE" or "FALL"
stmode = f"START_MODE_SW"   # Startmodes: "START_MODE_SW", "START_MODE_S_IN", or "START_MODE_FIRST_TRG"
dec_f = f"DECIM_FACTOR_1"   # Decimation factor: "DECIM_FACTOR_1" and "DECIM_FACTOR_2^n" where n can be integers 1-7
ext_clock = f"FALSE"        # If set "TRUE" the digitizer will take external clock via CLK-IN
ext_trg = f"FALSE"          # If set "TRUE" the external trigger participates to the global trigger generation in logic OR with the other enabled signals
                            # External trigger signal needs to be a square wave with V_pp <= 1.5V


# Group-specific Settings
gr_on = [0,1,2,3]                # Group numbers to participate in the event readout
gr_off = []                      # Group numbers to NOT participate in the event readout (all groups participate by default)
gr_ntrig_gen = []                # Groups NOT participating in global trigger generation (all groups participate by default)
gr_trig_out = []                 # Groups participating in TRG-OUT at GPO (all groups DO NOT participate by default)
gr_trig_lvl = [2600,2600,2600,2600] # in units of LSB (0-4095)
gr_dc = [i/40.96 for i in [2471,2368,2158,2458]] # in units of LSB (0-4095)
majlvl = 1                   # Group trigger majority level (integer from 0-3) 


# Channel-specific Settings
ch_on = [0, 10, 20, 30]                                                                                    # Channels to be recorded and TODO: displayed on live plot
ch_ntrig_gen = [1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,31]                # Channels NOT participating in global trigger generation
ch_dcoff = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]                                               # DC offset for individual channels

# Live histogram
lhist = True               # True or False (TODO: find an efficient way to implement an on/off for histogram feature)
binwidth = 3              # Time ticks

In [ ]:
with device.connect('dig1://caen.internal/usb') as dig:
    # Reset
    dig.cmd.Reset()
    dig.cmd.ClearData()

    handle = device.Node.get_node(dig)
    
    # Get board info
    nch = int(dig.par.NUMCH.value)
    n_analog_traces = int(dig.par.NUMANALOGTRACES.value)
    n_digital_traces = int(dig.par.NUMDIGITALTRACES.value)
    adc_samplrate_msps = float(dig.par.ADC_SAMPLRATE.value)  # in Msps
    adc_nbit = int(dig.par.ADC_NBIT.value)
    sampling_period_ns = int(1e3 / adc_samplrate_msps)
    fw_type = dig.par.FWTYPE.value


    # Configure digitizer
    dig.par.STARTMODE.value = stmode
    dig.par.RECLEN.value = f'{reclen}'
    dig.par.self_trigger_edge.value = trig_edge
    dig.par.posttrg.value = f'{post_trigger}'
    dig.par.decimation_factor.value = dec_f
    dig.par.dt_ext_clock.value = ext_clock
    dig.par.trg_ext_enable.value = ext_trg


    # Set group parameters
    for i in gr_off:
        dig.group[i].par.gr_enabled.value = f'FALSE'
    for i in gr_on:
        dig.group[i].par.gr_enabled.value = f'TRUE'
        dig.group[i].par.gr_threshold.value = f'{gr_trig_lvl[i]}'
        dig.group[i].par.gr_dcoffset.value = f'{gr_dc[i]}'
    for i in gr_ntrig_gen:
        dig.group[i].par.gr_trg_global_gen.value = f'FALSE'
    for i in gr_trig_out:
        dig.group[i].par.gr_out_propagate.value = f'TRUE'
    # Group trigger majority level
    ogtrg = device.Node.get_user_register(handle, c_uint32(0x810C))
    device.Node.set_user_register(handle, c_uint32(0x810C), c_uint32(ogtrg+majlvl*0b1000000000000000000000000))
    ntrg = device.Node.get_user_register(handle, c_uint32(0x810C))
    
    # Set channel parameters
    for i in ch_ntrig_gen:
        dig.ch[i].par.ch_trg_global_gen.value = f'FALSE'
    for i in range(len(ch_dcoff)):
        dig.ch[i].par.ch_dcoffset_corr.value = f'{ch_dcoff[i]}'

    
    # Compute record length in samples
    reclen_ns = int(dig.par.RECLEN.value)  # Read back RECLEN to check if there have been rounding
    reclen = int(reclen_ns / sampling_period_ns)


    # Set enabled endpoint to activate decode
    dig.endpoint.par.ActiveEndpoint.value = 'scope'

    # Configure endpoint
    data_format = [
        {
            'name': 'TIMESTAMP',
            'type': 'U64',
        },
        {
            'name': 'WAVEFORM',
            'type': 'U16',
            'dim': 2,
            'shape': [nch, reclen],
        },
        {
            'name': 'WAVEFORM_SIZE',
            'type': 'U64',
            'dim': 1,
            'shape': [nch],
        },
    ]


    # Store endpoint node
    decoded_endpoint_path = fw_type.strip('-')  # decoded endpoint path is just firmware type without -
    endpoint = dig.endpoint[decoded_endpoint_path]
    # set_read_data_format returns allocated buffers
    data = endpoint.set_read_data_format(data_format)

    timestamp = data[0].value
    waveform = data[1].value
    waveform_size = data[2].value
    
    # setup live plot
    fig, ax = plt.subplots(2)
    hfig = display(fig, display_id=True)
    amp = [[] for i in range(len(gr_on)*8)]
    
    # Start acquisition
    dig.cmd.ArmAcquisition()

    # Acquisition loop
    def update():
        wftemp = np.empty((len(ch_on),0))
        evtnum = 0
        # iterates for the desired amount of triggered events
        for _ in range(runtime):

            # Send software trigger
            dig.cmd.SendSwTrigger()

            # Wait for event
            try:
                dig.endpoint.scope.read_data(10, data)
                evtnum += 1
            except error.Error as ex:
                if ex.code == error.ErrorCode.TIMEOUT:
                    # Timeout expired, waiting again
                    continue
                elif ex.code == error.ErrorCode.STOP:
                    # End of run, exit the loop
                    print('End of run.')
                    break
                else:
                    # Other critical error, propagate it
                    raise ex
            
            # Save data
            wftemp=np.hstack((wftemp,np.vstack([waveform[i] for i in ch_on])))

            # Save designated amount of waveforms into one csv file
            if evtnum%filesize == 0:
                np.savetxt(f"wf{evtnum-filesize}-{evtnum-1}_{reclen}samples_per_evt.csv", wftemp, delimiter =", ", fmt ='% s')
                wftemp = np.empty((len(ch_on),0))
            else:
                pass

            colors = ["red", "blue", "peru", "green"]
            gr_plot = [0,10,20,30]
            # Change sampling rate of live plots
            #if _%1000 == 0:
            if _%1 == 0:
                for i in range(len(gr_on*8)):
                    if i in gr_plot:
                        # Collect triggered waveform amplitude (minimum value within a 400 time tick window at the center of each waveform)
                        amp[i].append(min(waveform[i][int(len(waveform[i])/2-200):int(len(waveform[i])/2+200)]))
                        ax[0].plot(np.arange(0, waveform_size[i]), waveform[i], color=colors[i//8])
                        ax[1].hist(amp[i], bins=range(min(amp[i]), max(amp[i]) + binwidth, binwidth), histtype="step", color=colors[i//8])

                #ax[0].set_ylim(1800,2100)
                #ax[1].set_xlim(1800,2100)
                # Used to display the common trigger level (the line statements need to be in the above for loop if unique trigger levels are used for different groups)
                ax[0].axhline(gr_trig_lvl[1], color="magenta")
                ax[1].axvline(gr_trig_lvl[1], color="magenta")
                fig.canvas.draw()
                hfig.update(fig)
                fig.canvas.flush_events()
                ax[0].clear()
                ax[1].clear()
                
        # Save the rest of the waveforms
        if len(wftemp[0]) != 0:
            np.savetxt(f"wf{evtnum-evtnum%filesize}-{evtnum-1}_{reclen}samples_per_evt.csv", wftemp, delimiter =", ", fmt ='% s')

    update()

    # Stop acquisition
    dig.cmd.DisarmAcquisition()

In [ ]:
# Note: right now the channels are triggering normally but the buffer is being filled up, for low rate studies (<10Hz) the current live plot feature is fine
# TODO: runtime control

In [ ]:
# Setup computer
# Change file writing format (1000evts per file, no zero channels, good naming scheme)